In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("6. Streaming Demo 1").getOrCreate()
df = spark.read.csv('fraud_detection.csv', header=True)
df.columns

22/10/10 11:42:30 WARN Utils: Your hostname, m0 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/10/10 11:42:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 11:42:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/10 11:42:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
df.show(5)

In [ ]:
df=df.drop("isFraud","isFlaggedFraud")

In [ ]:
df.show(5)

In [17]:
df.columns

['step',
 'type',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest']

In [ ]:
df.groupBy("step").count().show(5)

In [ ]:
%%time
steps = df.select("step").distinct().collect()
for step in steps[:]:
        temp_df = df.where(f"step = {step[0]}")
        temp_df.coalesce(1).write.mode("append").option("header","true").\
        csv("data/split_fraud_detection")

In [ ]:
!cd data/split_fraud_detection && ls

In [ ]:
part = spark.read.csv(
"data/split_fraud_detection/part-00000-d4a901de-5a84-40ba-8483-3c9888c657ad-c000.csv",
header=True,
inferSchema=True,

)

In [34]:
part.groupBy("step").count().show()

+----+-----+
|step|count|
+----+-----+
|   3|  552|
+----+-----+



In [35]:
dataSchema = part.schema

In [36]:
dataSchema

StructType([StructField('step', IntegerType(), True), StructField('type', StringType(), True), StructField('amount', DoubleType(), True), StructField('nameOrig', StringType(), True), StructField('oldbalanceOrg', DoubleType(), True), StructField('newbalanceOrig', DoubleType(), True), StructField('nameDest', StringType(), True), StructField('oldbalanceDest', DoubleType(), True), StructField('newbalanceDest', DoubleType(), True)])

In [37]:
streaming = (
spark.readStream.schema(dataSchema)
    .option("maxFilePerTrigger",1)
    .csv("data/split_fraud_detection")
)

In [43]:
dest_count=streaming.groupBy("nameDest").count().orderBy(F.desc("count"))

In [46]:
activityQuery=(
dest_count.writeStream.queryName("dest_counts")
.format("memory")
.outputMode("complete")
.start()

)

22/10/05 02:48:02 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d8888ad1-69aa-4215-b722-01987fcf16d2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/10/05 02:48:02 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [49]:
import time
for i in range(5):
    df_temp = spark.sql(
    "SELECT * FROM dest_counts WHERE nameDest!='nameDest' AND count>=2"    
    )
    if temp_df.count()>0:
        temp_df.show(10)
    time.sleep(0.5)
    
    
    

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|   8| PAYMENT|   2363.5|C1957951848|        11518|        9154.5|M1274315527|             0|             0|
|   8| PAYMENT|  1025.83|C1738171284|        21104|      20078.17| M959097834|             0|             0|
|   8| PAYMENT|   638.14|C1379495198|        20177|      19538.86|M1027223781|             0|             0|
|   8|CASH_OUT|373404.04| C151544341|       375640|       2235.96|C1284149862|             0|    2465823.51|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+



+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|   8| PAYMENT|   2363.5|C1957951848|        11518|        9154.5|M1274315527|             0|             0|
|   8| PAYMENT|  1025.83|C1738171284|        21104|      20078.17| M959097834|             0|             0|
|   8| PAYMENT|   638.14|C1379495198|        20177|      19538.86|M1027223781|             0|             0|
|   8|CASH_OUT|373404.04| C151544341|       375640|       2235.96|C1284149862|             0|    2465823.51|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+



+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|   8| PAYMENT|   2363.5|C1957951848|        11518|        9154.5|M1274315527|             0|             0|
|   8| PAYMENT|  1025.83|C1738171284|        21104|      20078.17| M959097834|             0|             0|
|   8| PAYMENT|   638.14|C1379495198|        20177|      19538.86|M1027223781|             0|             0|
|   8|CASH_OUT|373404.04| C151544341|       375640|       2235.96|C1284149862|             0|    2465823.51|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+



+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|   8| PAYMENT|   2363.5|C1957951848|        11518|        9154.5|M1274315527|             0|             0|
|   8| PAYMENT|  1025.83|C1738171284|        21104|      20078.17| M959097834|             0|             0|
|   8| PAYMENT|   638.14|C1379495198|        20177|      19538.86|M1027223781|             0|             0|
|   8|CASH_OUT|373404.04| C151544341|       375640|       2235.96|C1284149862|             0|    2465823.51|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+



+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|   8| PAYMENT|   2363.5|C1957951848|        11518|        9154.5|M1274315527|             0|             0|
|   8| PAYMENT|  1025.83|C1738171284|        21104|      20078.17| M959097834|             0|             0|
|   8| PAYMENT|   638.14|C1379495198|        20177|      19538.86|M1027223781|             0|             0|
|   8|CASH_OUT|373404.04| C151544341|       375640|       2235.96|C1284149862|             0|    2465823.51|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+



In [54]:
#Check if stream is active
spark.streams.active[0].isActive

True

In [55]:
activityQuery.status

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}